In [1]:
import pandas as pd

In [2]:
from vncorenlp import VnCoreNLP
annotator = VnCoreNLP(r"C:\Users\anhdq33\Downloads\VinBigData\ML\Project\2020_VBDI_BML\VnCoreNLP-master\VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [3]:
df = pd.read_csv('./data/data_clean_v3.csv').drop('Unnamed: 0', axis=1)[['NewsTitle', 'NewsFullContent']]
df.head()

,NewsTitle,NewsFullContent
0,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...
1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...
2,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...
3,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...
4,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...


In [4]:
dic = pd.read_csv('./data/fullTranslated.csv')['words'].values
len(dic)

23523

In [5]:
def word_segment(text):
    annotated_text = annotator.annotate(text)['sentences'][0]
    return [dic['form'].replace('_',' ').lower() for dic in annotated_text] 

In [6]:
df['word_segment'] = df['NewsFullContent'].apply(lambda text: word_segment(text))

In [7]:
ner = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

In [8]:
def vnner(text):
    annotated_text = ner.annotate(text)
    pos = []
    for dic in annotated_text['sentences'][0]:
        if dic['nerLabel'] != 'O':
            pos.append(dic['form'].replace('_',' ').lower())
        
    return list(set(pos))

In [9]:
df['ner'] = df['NewsFullContent'].apply(lambda x: vnner(x))

In [10]:
def tag(word, dic):
    if word in dic:
        return 1
    else:
        return 0

def label(row, dic):
    name_entity = row['ner']

    text = row['word_segment']
    keywords = list(set(text).intersection(dic)) + name_entity
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']

    text_df['tag'] = text_df['word'].apply(lambda x: tag(x, keywords))
    
#     return list(zip(text_df['word'], text_df['tag']))
    return sum(text_df['tag']), list(zip(text_df['word'], text_df['tag']))

In [11]:
name_entity_reg = []
for i,row in df.iterrows():
    name_entity_reg.append(label(row, dic))

In [12]:
df['filtered'] = name_entity_reg
df['match'], df['label'] = zip(*df.filtered.values)

In [13]:
data = df[df['match'] > 30][['NewsTitle','NewsFullContent','label']]

In [14]:
data.head()

,NewsTitle,NewsFullContent,label
0,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...,"[(ông, 0), (nguyễn hoà bình chủ tịch, 0), (hđq..."
1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...,"[(trong, 1), (khi, 0), (thị trường, 1), (châu ..."
2,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...,"[(kế hoạch, 1), (này, 0), (đã, 1), (được, 1), ..."
3,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...,"[(theo, 1), (đó, 0), (tháng, 1), (7/2009, 0), ..."
4,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,"[(trong, 1), (khi, 0), (chỉ, 1), (có, 1), (5, ..."


In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)
train.to_csv('./data/train_data.csv')
test.to_csv('./data/test_data.csv')

In [16]:
len(train)

535

In [17]:
len(test)

134

In [21]:
# newtest = pd.read_csv('./data/data_clean_outscope.csv').drop('Unnamed: 0', axis=1)[['NewsTitle', 'NewsFullContent']]
# newtest['word_segment'] = newtest['NewsFullContent'].apply(lambda text: word_segment(text))
# newtest['ner'] = newtest['NewsFullContent'].apply(lambda x: vnner(x))

In [23]:
name_entity_reg = []
for i,row in newtest.iterrows():
    name_entity_reg.append(label(row, dic))

newtest['filtered'] = name_entity_reg
newtest['match'], newtest['label'] = zip(*newtest.filtered.values)

In [27]:
newtest[newtest['match'] > 30][['NewsTitle','NewsFullContent','label']].to_csv('./data/newtest_data.csv')

In [29]:
len(newtest[newtest['match'] > 30])

86